In [31]:
from mxnet import autograd,nd, gluon
import d2l
true_w = nd.array([2, -3.4])
true_b = 4.2
def synthetic_data(w,b,num_examples):
    X = nd.random.normal(0,1,shape=(num_examples,len(w)))
    y = nd.dot(X,w)+b
    y+=nd.random.normal(scale=0.01,shape=y.shape)
    return X,y
features, labels = synthetic_data(true_w, true_b, 1000)

In [32]:
def load_array(data_arrays, batch_size, is_train=True):
    """Construct a Gluon data loader"""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [33]:
for X, y in data_iter:
    print(X, y)
    break


[[ 1.2900832  -1.3137891 ]
 [ 0.71653044 -1.9765451 ]
 [-1.7830178   0.86199874]
 [ 0.5227163  -0.04460207]
 [ 2.0832326  -0.21528225]
 [ 0.9540733   1.8686222 ]
 [-0.36880237 -0.07480539]
 [-1.2015084   0.2813507 ]
 [ 0.27240914  0.32651147]
 [-0.95514405  0.21987283]]
<NDArray 10x2 @cpu(0)> 
[11.237071   12.34144    -2.285279    5.3945584   9.093976   -0.24672414
  3.7228644   0.84910804  3.6297688   1.5461806 ]
<NDArray 10 @cpu(0)>


In [40]:
from mxnet.gluon import  nn
net = nn.Sequential()

In [41]:
net.add(nn.Dense(1))

In [42]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()

In [43]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.03})

In [44]:
num_epochs = 3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X),y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.042329
epoch 2, loss: 0.000157
epoch 3, loss: 0.000049


In [45]:
w = net[0].weight.data()
print(w-true_w.reshape(w.shape))


[[-0.00091302  0.00059366]]
<NDArray 1x2 @cpu(0)>


In [48]:
b = net[0].bias.data()
print(b - true_b)


[-0.00031233]
<NDArray 1 @cpu(0)>
